In [148]:
# %reset
from nsefolder2pickle import NseFolder2Pickle; 
import numpy as np
import pandas as pd

from nse_pickle_fns import *
import os

# import pandas_datareader as datareader
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
#from matplotlib.finance import candlestick_ohlc
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

# from talib import abstract
from talib.abstract import *
symbols_w=load_pkl(pickle_path+'symbols_to_work.pkl')
def percProfit(future_price,past_price):
    return (future_price-past_price)/past_price

    
def getstock_dict(symbol,datalimit):
    stock_array=np.array(load_pkl(pickle_path+symbol+'.pkl'))
    stock_array=stock_array[-datalimit:,:]
    input_arrays = {
        'date': mdates.date2num(stock_array[:,7]).astype(float),
        'open': stock_array[:,0].astype(float),
        'high': stock_array[:,1].astype(float),
        'low': stock_array[:,2].astype(float),
        'close': stock_array[:,3].astype(float),
        'volume': stock_array[:,5].astype(float),
        'ntrades': stock_array[:,8].astype(float)    }
    return input_arrays
def clipArray(s_arrays,nelements,at_where='end'): #you can give 'begin'
    if(at_where=='end'):
        for a in s_arrays:
            s_arrays[a]=s_arrays[a][0:-nelements]
    elif (at_where=='begin'):
        for a in s_arrays:
            s_arrays[a]=s_arrays[a][nelements:]
    else:
        print("Please specify at the 'end' or at 'begin' ")
    return s_arrays

def clipArrayNan(s_arrays,nelements,at_where='end'): #you can give 'begin'
    if(at_where=='end'):
        for a in s_arrays:
            s_arrays[a][-nelements:]=np.nan
    elif (at_where=='begin'):
        for a in s_arrays:
            s_arrays[a][0:nelements-1]=np.nan
    else:
        print("Please specify at the 'end' or at 'begin' ")
    return s_arrays
def remNanInstances(stock_arrays):
    b=np.zeros(len(stock_arrays['close']))
    for a in stock_arrays:
        b += np.isnan(stock_arrays[a])
    nonan_indx= b==0
    for a in stock_arrays:
        stock_arrays[a]=stock_arrays[a][nonan_indx]
    return stock_arrays
def show_arrays(stock_arrays,sample_to_show=10,at_where='begin'):
    if(at_where=='begin'):
        for a in stock_arrays:
            print(a,": ",stock_arrays[a][0:sample_to_show])
    else:
        for a in stock_arrays:
            print(a,": ",stock_arrays[a][-sample_to_show:])
    
    
def addTAProps(input_arrays):
    input_arrays['sma10']  = SMA(input_arrays, timeperiod=10) # calculate on close prices by default
    input_arrays['sma05']  = SMA(input_arrays, timeperiod=5) # calculate on close prices by default
    input_arrays['sma20']  = SMA(input_arrays, timeperiod=20) # calculate on close prices by default
    # output = SMA(input_arrays, timeperiod=25, price='open') # calculate on opens
    input_arrays['upper'] ,input_arrays['middle'] ,input_arrays['lower']  = BBANDS(input_arrays, 20, 2, 2)
    input_arrays['slowk'] ,input_arrays['slowd'] = STOCH(input_arrays, 5, 3, 0, 3, 0) # uses high, low, close by default
#     slowk, slowd = STOCH(input_arrays, 5, 3, 0, 3, 0, prices=['high', 'low', 'open'])
    return input_arrays
    
    
def addBuySell(stock_arrays,sample_dist=10,prop='close'):
    stk_array=stock_arrays[prop]
    stock_arrays['buy']=np.zeros(len(stk_array))
    stock_arrays['sell']=np.zeros(len(stk_array))
    stock_arrays['hold']=np.zeros(len(stk_array))
    stock_arrays['see']=np.zeros(len(stk_array))
    for j in range(0,len(stk_array)-sample_dist):
        if  percProfit(stk_array[sample_dist+j],stk_array[j])>0.2:
            stock_arrays['buy'][j]=1
        elif percProfit(stk_array[sample_dist+j],stk_array[j])<-0.1:
            stock_arrays['sell'][j]=1
        elif percProfit(stk_array[sample_dist+j],stk_array[j])>0.0:
            stock_arrays['hold'][j]=1
        else:
            stock_arrays['see'][j]=1
    return  cutArrays(stock_arrays,sample_dist)
# 
        
def cutArrays(s_arrays,nelements,at_where='end'): #you can give 'begin'
    main_arrays={}
    remain_arrays={}
#     for a in s_arrays:
#             main_arrays[a]=[]
#             remain_arrays[a]=[]
    if(at_where=='end'):
#         print('gone inside-----------------------')
        for a in s_arrays:
            main_arrays[a]=s_arrays[a][0:-nelements]
            remain_arrays[a]=s_arrays[a][-nelements:]
    elif (at_where=='begin'):
        for a in s_arrays:
            main_arrays[a]=s_arrays[a][-nelements:]
            remain_arrays[a]=s_arrays[a][0:nelements]
    else:
        print("Please specify at the 'end' or at 'begin' ")
    return main_arrays, remain_arrays        

In [149]:
datalimit=2000
stk_id=10
sampl_dist=10
ml_pickle_path='D:\SoftwareWebApps\Python\python_wb\Stocks\Data\ml_stk_pickle\\'
columns =['Open','High','Low','Close','Last','Volume','Value','Date','NTrades']
symbol=symbols_w[stk_id]
i=0
for symbol in symbols_w:
    i +=1
    print('Doing for {}. {} stock among {} sotocks is being done'.format(i,symbol,len(symbols_w)),end=': ')
    stock_arrays=getstock_dict(symbol,datalimit)
    stock_arrays=addTAProps(stock_arrays)
    ml_arrays,target_arrays=addBuySell(stock_arrays,sampl_dist)
    ml_arrays=remNanInstances(ml_arrays)
    print('{} rows'.format(len(ml_arrays['close'])))
    save_pkl(ml_pickle_path+symbol+'_ml.pkl',[ml_arrays,target_arrays])

Doing for 1. EQ_20MICRONS stock among 607 sotocks is being done: 1971 rows
Doing for 2. EQ_3IINFOTECH stock among 607 sotocks is being done: 1971 rows
Doing for 3. EQ_8KMILES stock among 607 sotocks is being done: 1032 rows
Doing for 4. EQ_A2ZINFRA stock among 607 sotocks is being done: 810 rows
Doing for 5. EQ_ABAN stock among 607 sotocks is being done: 1971 rows
Doing for 6. EQ_ABFRL stock among 607 sotocks is being done: 542 rows
Doing for 7. EQ_ACC stock among 607 sotocks is being done: 1971 rows
Doing for 8. EQ_ACE stock among 607 sotocks is being done: 1971 rows
Doing for 9. EQ_ADANIENT stock among 607 sotocks is being done: 1971 rows
Doing for 10. EQ_ADANIPORTS stock among 607 sotocks is being done: 1537 rows
Doing for 11. EQ_ADANIPOWER stock among 607 sotocks is being done: 1971 rows
Doing for 12. EQ_ADANITRANS stock among 607 sotocks is being done: 654 rows
Doing for 13. EQ_ADFFOODS stock among 607 sotocks is being done: 1867 rows
Doing for 14. EQ_ADVANIHOTR stock among 607 so

Doing for 113. EQ_COSMOFILMS stock among 607 sotocks is being done: 1971 rows
Doing for 114. EQ_COX&KINGS stock among 607 sotocks is being done: 1971 rows
Doing for 115. EQ_CPSEETF stock among 607 sotocks is being done: 987 rows
Doing for 116. EQ_CROMPTON stock among 607 sotocks is being done: 464 rows
Doing for 117. EQ_CUB stock among 607 sotocks is being done: 1971 rows
Doing for 118. EQ_CUMMINSIND stock among 607 sotocks is being done: 1971 rows
Doing for 119. EQ_CYIENT stock among 607 sotocks is being done: 958 rows
Doing for 120. EQ_DAAWAT stock among 607 sotocks is being done: 1971 rows
Doing for 121. EQ_DABUR stock among 607 sotocks is being done: 1971 rows
Doing for 122. EQ_DALMIABHA stock among 607 sotocks is being done: 1270 rows
Doing for 123. EQ_DALMIASUG stock among 607 sotocks is being done: 1728 rows
Doing for 124. EQ_DATAMATICS stock among 607 sotocks is being done: 1971 rows
Doing for 125. EQ_DBCORP stock among 607 sotocks is being done: 1971 rows
Doing for 126. EQ_DBL

Doing for 225. EQ_HINDALCO stock among 607 sotocks is being done: 1971 rows
Doing for 226. EQ_HINDCOPPER stock among 607 sotocks is being done: 1971 rows
Doing for 227. EQ_HINDOILEXP stock among 607 sotocks is being done: 1971 rows
Doing for 228. EQ_HINDPETRO stock among 607 sotocks is being done: 1971 rows
Doing for 229. EQ_HINDUNILVR stock among 607 sotocks is being done: 1971 rows
Doing for 230. EQ_HINDZINC stock among 607 sotocks is being done: 1971 rows
Doing for 231. EQ_HPL stock among 607 sotocks is being done: 376 rows
Doing for 232. EQ_HSCL stock among 607 sotocks is being done: 409 rows
Doing for 233. EQ_HTMEDIA stock among 607 sotocks is being done: 1971 rows
Doing for 234. EQ_HUBTOWN stock among 607 sotocks is being done: 1582 rows
Doing for 235. EQ_IBREALEST stock among 607 sotocks is being done: 1971 rows
Doing for 236. EQ_IBULHSGFIN stock among 607 sotocks is being done: 1151 rows
Doing for 237. EQ_IBVENTURES stock among 607 sotocks is being done: 747 rows
Doing for 238.

Doing for 335. EQ_M&M stock among 607 sotocks is being done: 1971 rows
Doing for 336. EQ_M&MFIN stock among 607 sotocks is being done: 1971 rows
Doing for 337. EQ_MADHUCON stock among 607 sotocks is being done: 1971 rows
Doing for 338. EQ_MADRASFERT stock among 607 sotocks is being done: 1971 rows
Doing for 339. EQ_MAGMA stock among 607 sotocks is being done: 1971 rows
Doing for 340. EQ_MAHABANK stock among 607 sotocks is being done: 1971 rows
Doing for 341. EQ_MAHINDCIE stock among 607 sotocks is being done: 1063 rows
Doing for 342. EQ_MAHLIFE stock among 607 sotocks is being done: 1971 rows
Doing for 343. EQ_MAHSEAMLES stock among 607 sotocks is being done: 1971 rows
Doing for 344. EQ_MAJESCO stock among 607 sotocks is being done: 641 rows
Doing for 345. EQ_MANALIPETC stock among 607 sotocks is being done: 1971 rows
Doing for 346. EQ_MANAPPURAM stock among 607 sotocks is being done: 1920 rows
Doing for 347. EQ_MANINDS stock among 607 sotocks is being done: 1971 rows
Doing for 348. EQ

Doing for 447. EQ_QUICKHEAL stock among 607 sotocks is being done: 528 rows
Doing for 448. EQ_RADICO stock among 607 sotocks is being done: 1971 rows
Doing for 449. EQ_RAIN stock among 607 sotocks is being done: 1140 rows
Doing for 450. EQ_RAJESHEXPO stock among 607 sotocks is being done: 1971 rows
Doing for 451. EQ_RAJSREESUG stock among 607 sotocks is being done: 1971 rows
Doing for 452. EQ_RALLIS stock among 607 sotocks is being done: 1971 rows
Doing for 453. EQ_RAMCOCEM stock among 607 sotocks is being done: 1115 rows
Doing for 454. EQ_RAMCOSYS stock among 607 sotocks is being done: 1971 rows
Doing for 455. EQ_RAMKY stock among 607 sotocks is being done: 1785 rows
Doing for 456. EQ_RAYMOND stock among 607 sotocks is being done: 1971 rows
Doing for 457. EQ_RBLBANK stock among 607 sotocks is being done: 398 rows
Doing for 458. EQ_RCF stock among 607 sotocks is being done: 1971 rows
Doing for 459. EQ_RCOM stock among 607 sotocks is being done: 1971 rows
Doing for 460. EQ_RECLTD stock 

Doing for 556. EQ_TORNTPOWER stock among 607 sotocks is being done: 1971 rows
Doing for 557. EQ_TRIDENT stock among 607 sotocks is being done: 1713 rows
Doing for 558. EQ_TRIGYN stock among 607 sotocks is being done: 1971 rows
Doing for 559. EQ_TRITURBINE stock among 607 sotocks is being done: 1582 rows
Doing for 560. EQ_TRIVENI stock among 607 sotocks is being done: 1971 rows
Doing for 561. EQ_TTML stock among 607 sotocks is being done: 1971 rows
Doing for 562. EQ_TV18BRDCST stock among 607 sotocks is being done: 1649 rows
Doing for 563. EQ_TVSMOTOR stock among 607 sotocks is being done: 1971 rows
Doing for 564. EQ_TWL stock among 607 sotocks is being done: 1971 rows
Doing for 565. EQ_UBL stock among 607 sotocks is being done: 1971 rows
Doing for 566. EQ_UCOBANK stock among 607 sotocks is being done: 1971 rows
Doing for 567. EQ_UFLEX stock among 607 sotocks is being done: 1971 rows
Doing for 568. EQ_UJAAS stock among 607 sotocks is being done: 1083 rows
Doing for 569. EQ_UJJIVAN stock

In [146]:
# ml_arrays,target_arrays
len(symbols_w)
# stock_arrays['close'][0:-10],stock_arrays['close'][-10:]

607

In [142]:

# ml_arrays,target_arrays
show_arrays(ml_arrays,12)


date :  [733909. 733910. 733911. 733912. 733913. 733916. 733917. 733918. 733919.
 733920. 733923. 733924.]
open :  [125.   120.   120.1  117.5  117.2  117.4  117.1  116.   117.45 117.
 119.   120.6 ]
high :  [125.   123.5  120.4  120.1  117.2  119.2  117.1  118.   117.45 119.9
 122.   121.7 ]
low :  [118.5  119.5  116.5  117.3  112.   116.6  114.   114.1  115.1  117.
 117.05 118.4 ]
close :  [120.55 120.5  117.2  118.05 115.9  117.   114.4  117.45 116.1  119.35
 121.6  118.95]
volume :  [296879. 342992. 363505. 570840. 263701. 287198. 261058. 618486. 196327.
 383665. 378686. 204292.]
ntrades :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
sma10 :  [121.01  120.93  120.165 119.7   119.585 119.315 118.635 118.395 117.87
 117.65  117.755 117.6  ]
sma05 :  [120.9  120.76 120.23 119.57 118.44 117.73 116.51 116.56 116.17 116.86
 117.78 118.69]
sma20 :  [120.9525 121.1525 120.985  120.92   120.77   120.6275 120.35   120.27
 119.895  119.6025 119.3825 119.265 ]
upper :  [125.85662836 125.62088617 125

In [143]:
# pd.DataFrame(ml_arrays)
# for a in ml_arrays:
#     b = ml_arrays[a]
#     print(a,": ",b)
ml,t=load_pkl(ml_pickle_path+symbols_w[stk_id]+'_ml.pkl')

In [144]:
ml,t

({'buy': array([0., 0., 0., ..., 0., 0., 0.]),
  'close': array([120.55, 120.5 , 117.2 , ...,  24.65,  24.55,  24.4 ]),
  'date': array([733909., 733910., 733911., ..., 736822., 736823., 736824.]),
  'high': array([125.  , 123.5 , 120.4 , ...,  25.3 ,  24.75,  24.65]),
  'hold': array([1., 0., 1., ..., 0., 0., 0.]),
  'low': array([118.5 , 119.5 , 116.5 , ...,  24.35,  24.4 ,  23.95]),
  'lower': array([116.04837164, 116.68411383, 116.19896824, ...,  24.3105592 ,
          24.22628786,  24.17479029]),
  'middle': array([120.9525, 121.1525, 120.985 , ...,  25.495 ,  25.4025,  25.2925]),
  'ntrades': array([    0.,     0.,     0., ..., 19998., 14882., 10627.]),
  'open': array([125.  , 120.  , 120.1 , ...,  25.2 ,  24.5 ,  24.65]),
  'see': array([0., 1., 0., ..., 0., 0., 0.]),
  'sell': array([0., 0., 0., ..., 1., 1., 1.]),
  'slowd': array([64.66873922, 58.28194953, 44.02231373, ..., 39.75213487,
         35.63889092, 30.56039723]),
  'slowk': array([61.70990287, 45.47263682, 24.884401

In [92]:
stock_arrays[1]

KeyError: 1

In [77]:
# type(stock_arrays)
stock_arrays['close'][b==0]

array([60.2 , 61.75, 61.65, 62.95, 63.8 , 63.7 , 64.35, 63.45, 61.85,
       62.  , 58.9 , 58.75, 60.35, 59.05, 58.5 , 58.8 , 58.95, 60.  ,
       60.7 , 60.35, 60.35, 62.55, 63.55, 65.35, 65.5 , 65.55, 65.5 ,
       59.85, 57.7 , 59.5 , 56.45, 57.15, 58.15, 59.5 , 61.45, 60.55,
       60.4 , 60.  , 59.2 , 57.9 , 56.15, 56.05, 57.25, 56.6 , 55.5 ,
       56.2 , 56.35, 56.75, 55.6 , 53.3 , 54.1 , 54.1 , 52.  , 53.55,
       54.  , 55.1 , 55.8 , 55.35, 55.  , 52.8 , 49.4 , 48.7 , 47.2 ,
       50.2 , 50.7 , 49.3 , 49.35, 49.  , 50.2 , 50.75, 50.  , 51.15,
       50.4 , 50.5 , 48.6 , 49.2 , 53.  , 51.3 , 50.45, 47.7 , 46.7 ,
       42.45, 45.1 , 44.4 , 43.4 , 43.15, 41.95, 42.65, 44.2 , 42.9 ,
       43.35, 43.25, 42.2 , 42.6 , 43.6 , 44.9 , 45.45, 45.9 , 45.4 ,
       45.8 , 46.05, 45.65, 48.  , 49.15, 47.7 , 47.75, 46.45, 46.6 ,
       46.4 , 45.15, 45.85, 47.25, 48.2 , 46.35, 46.  , 45.45, 46.55,
       45.25, 46.1 , 45.75, 45.6 , 44.35, 44.45, 45.6 , 44.65, 44.5 ,
       44.85, 43.6 ,

In [23]:
stock_arrays['buy'],stock_arrays['sell']

TypeError: 'NoneType' object is not subscriptable